In [6]:
import sys
import numpy as np
import cv2


# 비디오 파일 열기
cap = cv2.VideoCapture('shift.mp4')

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

In [7]:
# 초기 사각형 영역: (x, y, w, h)
x, y, w, h = 500, 500, 100, 180
rc = (x, y, w, h)

ret, frame = cap.read()

if not ret:
    print('frame read failed!')
    sys.exit()

roi = frame[y:y+h, x:x+w]
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

In [8]:
# HS 히스토그램 계산
channels = [0, 1]
ranges = [0, 180, 0, 256]
hist = cv2.calcHist([roi_hsv], channels, None, [90, 128], ranges)


In [9]:
# Mean Shift 알고리즘 종료 기준
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [10]:
# 비디오 매 프레임 처리
while True:
    ret, frame = cap.read()

    if not ret:
        break

    # HS 히스토그램에 대한 역투영
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproj = cv2.calcBackProject([frame_hsv], channels, hist, ranges, 1)

    # Mean Shift
    _, rc = cv2.meanShift(backproj, rc, term_crit) #역투영 영상, 영역결과, 알고리즘 종료 기준

    # 추적 결과 화면 출력
    cv2.rectangle(frame, rc, (0, 0, 255), 2)
    cv2.imshow('frame', frame)
    cv2.imshow('hist', hist) #hs 히스토그램
    cv2.imshow('backproj', backproj) # 역투영


    if cv2.waitKey(60) == 27:
        break

cap.release()
cv2.destroyAllWindows()